In [22]:
import requests
import json
import os
MY_API_KEY='9eab72c1b16e7eabbcc3b4cceb7851e7'


def write_data(new_data):
    with open('data.json','r') as json_file: 
        if(os.stat("data.json").st_size !=0):
            data_json = json.load(json_file)
            data_json.append(new_data["search-results"]["entry"])
        else:
            data_json=new_data["search-results"]["entry"]    
    with open('data.json', 'w') as outfile:
        json.dump(data_json, outfile)

def get_chunk(key,start,count='25',date='2010-2020'):
    resp = requests.get(f"https://api.elsevier.com/content/search/scopus?query={key}&start={start}&count={count}&date={date}",
                    headers={'Accept':'application/json',
                             'X-ELS-APIKey': MY_API_KEY})
    data=json.loads(resp.text.encode('utf-8'))
    return data

def main():        
        data=get_chunk("TITLE%28management+AND+smart+production%29",start=0,count=25)
        print(data)
        #total_results=int(data['search-results']['opensearch:totalResults'])
        #print(f'total results {total_results}')
        #i=25
        #write_data(data)
        #while(i<total_results):
         #   data=get_chunk("KEY%28industry+4.0%29",start=i)
         #   write_data(data)
         #   print(i)
         #   i=i+25
            

if __name__ == "__main__":
    main()



{'search-results': {'opensearch:totalResults': '42', 'opensearch:startIndex': '0', 'opensearch:itemsPerPage': '25', 'opensearch:Query': {'@role': 'request', '@searchTerms': 'TITLE(management AND smart production)', '@startPage': '0'}, 'link': [{'@_fa': 'true', '@ref': 'self', '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=TITLE%28management+AND+smart+production%29&date=2010-2020', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'first', '@href': 'https://api.elsevier.com/content/search/scopus?start=0&count=25&query=TITLE%28management+AND+smart+production%29&date=2010-2020', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'next', '@href': 'https://api.elsevier.com/content/search/scopus?start=25&count=25&query=TITLE%28management+AND+smart+production%29&date=2010-2020', '@type': 'application/json'}, {'@_fa': 'true', '@ref': 'last', '@href': 'https://api.elsevier.com/content/search/scopus?start=17&count=25&query=TITLE%28management+AND+smart+pr

In [131]:
188584+167553

356137